In [1]:
import numpy as np
import matplotlib
import pandas as pd
import PIL
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from tqdm import tqdm

import os
import random

In [23]:
def make_easy_captcha(text, output_dir="output", font_path="arial.ttf", font_size=12, 
                  text_color=(0, 0, 0), bg_color=(255, 255, 255), padding=20):
    """
    Render text strings as images with white background and consistent font.
    
    Args:
        texts (list): List of text strings to convert to images
        output_dir (str): Output directory to save images
        font_path (str): Path to font file (.ttf)
        font_size (int): Font size in points
        text_color (tuple): Text color as RGB tuple
        bg_color (tuple): Background color as RGB tuple
        padding (int): Padding around text in pixels
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the font
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        raise Exception(f"Font file not found at {font_path}.")
    
    # Calculate text size using the font
    text_width, text_height = ImageDraw.Draw(Image.new('RGB', (1, 1))).textsize(text, font=font)

    # Create image with padding
    image_width = text_width + 2 * padding
    image_height = text_height + 2 * padding
    image = Image.new('RGB', (image_width, image_height), bg_color)
    draw = ImageDraw.Draw(image)

    # Draw text centered on image
    draw.text((padding, padding), text, font=font, fill=text_color)

    # Save image
    filename = os.path.join(output_dir, f"{text.lower().replace(' ', '_')}.png")
    
    return image, text


# Example usage
# sample_words = ["Hello", "World", "Python", "OpenAI"]

# text_to_image(
#     texts=sample_words,
#     output_dir="easy_set",
#     font_path="/home/swapnanil_mukherjee/captcha/fonts/Arial.ttf",
#     font_size=55,
#     padding=50
# )

In [24]:
def randomize_capitalization(text):
    """Randomly capitalize each character in the text"""
    return ''.join([c.upper() if random.random() < 0.5 else c.lower() for c in text])

def add_noise_overlay(image, intensity=50):
    """Optimized noise overlay using NumPy vectorization"""
    # Convert image to NumPy array (uint8 -> int32 to prevent overflow)
    arr = np.array(image).astype(np.int32)
    
    # Generate random noise in one operation
    noise = np.random.randint(-intensity, intensity+1, 
                            size=arr.shape, dtype=np.int32)
    
    # Add noise and clip values to maintain the pixel value range 
    noisy = np.clip(arr + noise, 0, 255).astype(np.uint8)
    
    # Convert back to PIL Image
    noisy_image = Image.fromarray(noisy)
    
    return noisy_image.filter(ImageFilter.GaussianBlur(radius=1))


def make_hard_captcha(text, output_dir="noisy_output", fonts=["arial.ttf"], 
                      font_size=32, padding=20, noise_intensity=40):
    """
    Create images with:
    - Text first on plain white background
    - Noise overlay applied to entire image
    - Random capitalization
    - Random text colors
    - Random font selection
    """
    os.makedirs(output_dir, exist_ok=True)
    font_dir = '/home/swapnanil_mukherjee/captcha/fonts'

    processed_text = randomize_capitalization(text)

    # Random font selection
    font_face = random.choice(fonts)
    font_path = os.path.join(font_dir, font_face)

    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        raise Exception(f"Font file not found: {font_path}")

    # Calculate text dimensions
    dummy_draw = ImageDraw.Draw(Image.new('RGB', (1, 1)))
    text_width, text_height = dummy_draw.textsize(processed_text, font=font)

    # Create image dimensions
    image_width = text_width + 2 * padding
    image_height = text_height + 2 * padding

    # Create plain white background
    img = Image.new('RGB', (image_width, image_height), (255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Generate random text color
    text_color = (random.randint(0,255), random.randint(0,255), random.randint(0,255))

    # Draw text first on clean background
    draw.text((padding, padding), processed_text, font=font, fill=text_color)

    # Add noise overlay to entire image (background + text)
    img = add_noise_overlay(img, noise_intensity)

    # Save file
    filename = f"{text.lower().replace(' ', '_')}.png"
    
    return img, processed_text

# Example usage (keep the same as before)
fonts = ['Times New Roman.ttf',
         'Lucida Console.ttf',
         'DancingScript-VariableFont_wght.ttf',
         'Courier New.ttf',
         'Verdana.ttf',
         'Tahoma.ttf',
         'Comic Sans MS.ttf',
         'impact.ttf',
         'georgia.ttf',
         'Arial.ttf']

# sample_words = ["This", "Hard", "Set", "Contrast"]

# noisy_text_to_image(
#     texts=sample_words,
#     fonts=fonts,
#     output_dir="hard_set",
#     font_size=55,
#     padding=50,
#     noise_intensity=75 
# )

In [25]:
def make_hard_captcha_bonus(text, output_dir="bonus", fonts=["arial.ttf"], 
                     font_size=32, padding=20, noise_intensity=40):
    """
    Create CAPTCHA images with:
    - Green background: normal text
    - Red background: reversed text
    - Random capitalization for both cases
    - Random text color
    - Random font selection
    - Noise overlay
    """
    os.makedirs(output_dir, exist_ok=True)
    font_dir = '/home/swapnanil_mukherjee/captcha/fonts'

    # Randomly choose background color (50/50 chance)
    background_color = random.choice(['green', 'red'])
    base_color = (0, 128, 0) if background_color == 'green' else (255, 0, 0)

    # Process text with random capitalization
    processed_text = randomize_capitalization(text)
    label = processed_text
    
    # Reverse text for red background
    if background_color == 'red':
        processed_text = processed_text[::-1]

    # Random font selection
    font_face = random.choice(fonts)
    font_path = os.path.join(font_dir, font_face)

    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        raise Exception(f"Font file not found: {font_path}")

    # Calculate text dimensions
    dummy_draw = ImageDraw.Draw(Image.new('RGB', (1, 1)))
    text_width, text_height = dummy_draw.textsize(processed_text, font=font)

    # Create image dimensions
    image_width = text_width + 2 * padding
    image_height = text_height + 2 * padding

    # Create colored background
    img = Image.new('RGB', (image_width, image_height), base_color)
    draw = ImageDraw.Draw(img)

    # Generate random text color (ensure visibility)
    text_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    # Draw text
    draw.text((padding, padding), processed_text, font=font, fill=text_color)

    # Add noise overlay
    img = add_noise_overlay(img, noise_intensity)

    # Save file
    filename = f"{text.lower().replace(' ', '_')}_{background_color}.png"
    # img.save(os.path.join(output_dir, filename))
    
    return img, label

In [31]:
with open('words.txt', 'r') as f:
    content = f.read()

words = content.split(',')
print(len(words))

742


In [27]:
num_samples = 50000     # number of samples to be generated in EACH set 
word_list = words # take all (200) words from the list of all words

fonts = ['Times New Roman.ttf',
         'Lucida Console.ttf',
         'DancingScript-VariableFont_wght.ttf',
         'Courier New.ttf',
         'Verdana.ttf',
         'Tahoma.ttf',
         'Comic Sans MS.ttf',
         'impact.ttf',
         'georgia.ttf',
         'Arial.ttf']

easy_mapping = {}

# generate 'num_samples' number of easy examples   
for i in tqdm(range(num_samples)):
    word = random.choice(word_list)
    
    img, label = make_easy_captcha(
     text=word,
     output_dir="easy_set",
     font_path="/home/swapnanil_mukherjee/captcha/fonts/Arial.ttf", # constant font -> Arial
     font_size=55,
     padding=50
 )
    
    filename = f"{i}.png"
    
    img.save(os.path.join("easy_set", filename), dpi=(700, 700))
    easy_mapping[filename] = label
    
easy_set_df = pd.DataFrame(easy_mapping.items(), columns=['image', 'label'])
easy_set_df.to_csv("easy_set_labels.csv")

hard_mapping = {}
    
# generate 'num_samples' number of hard examples 
for i in tqdm(range(num_samples)):
    word = random.choice(word_list)
    
    img, label = make_hard_captcha(
    text=word,
    fonts=fonts,                # randomly choose font from a given list of fonts
    output_dir="hard_set",
    font_size=55,
    padding=50,
    noise_intensity=75 
)
    
    filename = f"{i}.png"
    
    img.save(os.path.join("hard_set", filename), dpi=(700, 700))
    hard_mapping[filename] = label

hard_set_df = pd.DataFrame(hard_mapping.items(), columns=['image', 'label'])
hard_set_df.to_csv("hard_set_labels.csv")

100%|██████████| 50000/50000 [14:34<00:00, 57.18it/s]


In [28]:
# generate 'num_samples' number of hard examples 
num_samples = 100000     # generate 100K bonus samples
word_list = words # take all words from the list of all (200) words

fonts = ['Times New Roman.ttf',
         'Lucida Console.ttf',
         'DancingScript-VariableFont_wght.ttf',
         'Courier New.ttf',
         'Verdana.ttf',
         'Tahoma.ttf',
         'Comic Sans MS.ttf',
         'impact.ttf',
         'georgia.ttf',
         'Arial.ttf']


bonus_mapping = {}

for i in tqdm(range(num_samples)):
    word = random.choice(words)
    
    img, label = make_hard_captcha_bonus(
    text=word,
    fonts=fonts,                # randomly choose font from a given list of fonts
    output_dir="bonus_set",
    font_size=55,
    padding=50,
    noise_intensity=75 
)
    
    filename = f"{i}.png"
    
    img.save(os.path.join("bonus_set", filename), dpi=(700, 700))
    bonus_mapping[filename] = label

bonus_set_df = pd.DataFrame(bonus_mapping.items(), columns=['image', 'label'])
bonus_set_df.to_csv("bonus_set_labels.csv")

100%|██████████| 100000/100000 [26:08<00:00, 63.74it/s]
